In [ ]:
%run dataset.ipynb
%run models.ipynb
import time
import random
import numpy as np
from glob import glob
from tqdm import tqdm
from operator import add
from PIL import Image
import segmentation_models_pytorch as smp
import os 

import re
import torch.optim as optim

import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
import import_ipynb
import json
from pyfiles.efficientunet import *



In [ ]:

# Load the JSON file
with open('dict_indices.json', 'r') as file:
    data = json.load(file)
train_indices = data['train_indices']
val_indices = data['val_indices']

with open('domainb_indices.json', 'r') as file:
    data = json.load(file)

domainb_indices = data['train']


train_path = os.path.join(os.getcwd(),"train")
mask_path = os.path.join(os.getcwd(),"train_gt")
train_x = os.listdir('train')
train_x.sort(key=extract_numerical_part)
train_y = os.listdir('train_gt')
train_y.sort(key=extract_numerical_part)

val_path = os.path.join(os.getcwd(),"val")
val_gt = os.path.join(os.getcwd(),"val_gt")
val_x = os.listdir('val')
val_x.sort(key=extract_numerical_part)
val_y = os.listdir('val_gt')
val_y.sort(key=extract_numerical_part)

testB = os.path.join(os.getcwd(),"imgs")
test_gtB = os.path.join(os.getcwd(),"Gts")
path_soft_gtB  =  os.path.join(os.getcwd(),"soft_gtB")

testB_images = os.listdir(testB)
testB_images.sort(key=extract_numerical_part)
testB_masks = os.listdir(test_gtB)
testB_masks.sort(key=extract_numerical_part)



# vss = dess400(train_path,mask_path,train_indices,train_transform)
# vall = dess400(val_path,val_gt,val_indices,valid_transform)
# dbb = dess400(testB,test_gtB,domainb_indices,train_transform)

vs= Dess2(train_path,mask_path,train_x ,train_y , train_transform)

val= Dess2(val_path , val_gt, val_x,val_y , valid_transform)
domainb = hotencoded(testB , test_gtB , testB_images    , testB_masks , num_classes=7,transform=valid_transform)
# domainb_soft = hot400(testB , path_soft_gtB, domainb_indices, num_classes=7,transform=train_transform)
domainb_small = hot400(testB , test_gtB,domainb_indices,num_classes=7,transform=valid_transform)
ds = hot400(train_path , mask_path,train_indices,num_classes=7,transform=train_transform)
ds_val = hot400(val_path , val_gt , val_indices,num_classes=7,transform=valid_transform)
# ds = hotencoded(train_path,mask_path , train_x,train_y , num_classes=7,transform=train_transform)
# ds_val = hotencoded(val_path,val_gt,val_x,val_y,num_classes=7,transform=valid_transform)
dbtrain,dbtest = splitset(testB_masks,0.8)
dbtrain_l,dbtrain_ul = splitset(dbtrain,0.10)
domainb_train_l = hot400(testB , test_gtB,dbtrain_l,num_classes=7,transform=train_transform)
domainb_train_ul = hot400(testB , test_gtB,dbtrain_ul,num_classes=7,transform=train_transform)
domainb_test = hot400(testB,test_gtB , dbtest,num_classes=7,transform=valid_transform)


In [ ]:
a=torch.load("/home/khan/Desktop/ammar/EfficientUnet-PyTorch-master/ftmap")

In [ ]:
a.shape

In [ ]:
domainb_test[0][2].unique()

In [ ]:
len(domainb_test),len(domainb_train_l),len(domainb_train_ul)

In [ ]:
splitset

In [ ]:
ll=[]
for x in domainb_train_l:
    _,_,mask,_=x
    ll.append(np.unique(mask))

In [ ]:
count=0
for x in ll:
    if len(x) >1:
        count=count+1

In [ ]:
count/len(ll)

In [ ]:
# Define the loss function for knowledge distillation
def distillation_loss(outputs, labels, teacher_outputs, temperature):
    soft_logits = nn.functional.log_softmax(outputs / temperature, dim=1)
    soft_targets = nn.functional.softmax(teacher_outputs / temperature, dim=1)
    return nn.functional.kl_div(soft_logits, soft_targets, reduction='batchmean')

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs



In [ ]:
len(dbtrain_l)

In [ ]:
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

teacher_model = smp.Unet(
    encoder_name="efficientnet-b4",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=7,                      # model output channels (number of classes in your dataset)
)
teacher_model.load_state_dict(torch.load("efb4-full.pth" , map_location=torch.device(device)))
# teacher_model = get_efficientunet_b4(out_channels=7, concat_input=True, pretrained=True)
# teacher_model.load_state_dict(torch.load("efb4-full.pth",map_location = torch.device(device)))
# student_model = get_efficientunet_b4(out_channels=7, concat_input=True, pretrained=True)
# student_model.load_state_dict(torch.load("efb4-full.pth",map_location = torch.device(device)))
student_model = smp.Unet(
    
    encoder_name="efficientnet-b4",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=7,                      # model output channels (number of classes in your dataset)
)
student_model.load_state_dict(torch.load("efb4-full.pth",map_location = torch.device(device)))

In [ ]:
def softmax_mse_loss(input_logits, target_logits):
    """Takes softmax on both sides and returns MSE loss

    Note:
    - Returns the sum over all examples. Divide by the batch size afterwards
      if you want the mean.
    - Sends gradients to inputs but not the targets.
    """
    assert input_logits.size() == target_logits.size(), "INPUT_LOGITS TARGET_LOGITS MISMATCH"
    input_softmax = F.softmax(input_logits, dim=1)
    target_softmax = F.softmax(target_logits, dim=1)
    #logging.info("input_softmax"+str(input_softmax.shape))
    #logging.info("target_softmax"+str(target_softmax.shape))
    mse_loss = (input_softmax - target_softmax) ** 2
    return mse_loss

def sigmoid_rampup(current, rampup_length):
    """Exponential rampup from https://arxiv.org/abs/1610.02242"""
    if rampup_length == 0:
        return 1.0
    else:
        current = np.clip(current, 0.0, rampup_length)
        phase = 1.0 - current / rampup_length
        return float(np.exp(-5.0 * phase * phase))

def get_current_consistency_weight(epoch):
    # Consistency ramp-up from https://arxiv.org/abs/1610.02242
    consistency = 0.1
    consistency_rampup = 40.0
    return consistency * sigmoid_rampup(epoch, consistency_rampup)
    
def update_ema_variables(model, ema_model, alpha, global_step):
    # Use the true average until the exponential average is more correct
    alpha = min(1 - 1 / (global_step + 1), alpha)
    for ema_param, param in zip(ema_model.parameters(), model.parameters()):
        ema_param.data.mul_(alpha).add_(1 - alpha, param.data)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# # without SEN --ORIGNAL
    
# def train_mean2d(model, ema_model ,labelled_loader,unlabelled_loader, focal_loss ,optimizer ,iter_num , scheduler, device):
#     epoch_loss = 0
#     count = 0

#     model.train()
    
#     for i, data in enumerate(zip(labelled_loader ,unlabelled_loader) ):

        
#         data1, data2 = data
#         images, masks,_,_ = data1
#         images, masks= images.to(device), masks.to(device)
#         outputs = model(images.float())
#         outputs_soft = torch.softmax(outputs, dim=1)

        

#         unlabeled_images = data2[0]
#         unlabeled_images = unlabeled_images.to(device)
#         unlabeled_outputs = model(unlabeled_images.float())
        
#         noise = torch.clamp(torch.randn_like(unlabeled_images.float()) *  0.1, -0.2, 0.2)
#         ema_inputs = unlabeled_images + noise

#         with torch.no_grad():
#                 ema_output = ema_model(ema_inputs.float())
#                 ema_output_soft = torch.softmax(ema_output, dim=1)
        
#         # loss_ce = ce_loss(outputs, masks.float())
#         # loss_dice = dice_loss(outputs_soft, masks.unsqueeze(1))
#         supervised_loss = focal_loss(outputs,masks)
#         consistency_weight = get_current_consistency_weight(iter_num // consistency_rampup)
#         if iter_num < 400:
#             consistency_loss = 0.0
#         else:
#             consistency_loss = torch.mean((outputs_soft - ema_output_soft) ** 2)
#         loss = supervised_loss + consistency_weight * consistency_loss

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         # scheduler.step()
        
#         update_ema_variables(model, ema_model, ema_decay, iter_num)
        
#         iter_num = iter_num + 1

#         epoch_loss += loss.item() 
    
#     epoch_loss = epoch_loss/len(train_loader)
#     return epoch_loss
    

In [ ]:
def train_mean2d(model, ema_model, labelled_loader, unlabelled_loader, focal_loss, optimizer, iter_num, scheduler, device):
    epoch_loss = 0
    count = 0
    prev_supernoise = None

    model.train()

    for i, data in enumerate(zip(labelled_loader, unlabelled_loader)):

        data1, data2 = data
        images, masks, _, _ = data1
        images, masks = images.to(device), masks.to(device)

        if prev_supernoise is not None:
            s_noise = torch.mean(prev_supernoise, dim=1, keepdim=True)
            target_size = images.shape[2:]
            s_noise = F.interpolate(s_noise, size=target_size, mode='nearest')
            s_noise = s_noise.expand(-1, 3, -1, -1)
            outputs, _ = model(images.float() + s_noise)
        else:
            outputs, _ = model(images.float())

        outputs_soft = torch.softmax(outputs, dim=1)

        unlabeled_images = data2[0]
        unlabeled_images = unlabeled_images.to(device)

        unlabeled_outputs, _, bases1 = model(unlabeled_images.float())

        noise = torch.clamp(torch.randn_like(unlabeled_images.float()) * 0.1, -0.2, 0.2)
        ema_inputs = unlabeled_images + noise

        with torch.no_grad():
            ema_output, supernoise, bases2 = ema_model(ema_inputs.float())
            ema_output_soft = torch.softmax(ema_output, dim=1)

        # Update the supernoise for the next iteration
        prev_supernoise = supernoise

        supervised_loss = focal_loss(outputs, masks)

        consistency_weight = get_current_consistency_weight(iter_num // consistency_rampup)
        if iter_num < 1200:
            consistency_loss = 0.0
        else:
            consistency_loss = torch.mean((outputs_soft - ema_output_soft) ** 2)

        # Add low-rank subspace loss
        J_s = bases1.transpose(2, 1)  # Transpose bases1 to match the shape of J_t
        J_t = bases2
        lrs_loss = (2 * J_s.shape[0] - 2 * torch.norm(torch.matmul(J_s.transpose(1, 2), J_t), 'fro')**2) / J_s.shape[0]

        loss = supervised_loss + consistency_weight * consistency_loss + lrs_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        update_ema_variables(model, ema_model, ema_decay, iter_num)

        iter_num = iter_num + 1

        epoch_loss += loss.item()

    epoch_loss = epoch_loss / len(train_loader)
    return epoch_loss

In [ ]:
# Define optimizer and criterion

checkpoint_path = "."
optimizer = optim.Adam(student_model.parameters(), lr=0.001)  #1-^3(best) 
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.9)
loss_fn = CategoricalFocalLoss(gamma=2.0, alpha=None, reduction='mean')
consistency_criterion = softmax_mse_loss
# criterion  = CategoricalFocalLoss(gamma=2.0, alpha=None, reduction='mean')

# Transfer models to device
student_model.to(device)
teacher_model.to(device)
# old_teacher_model.load_state_dict(torch.load("efb4-full.pth",map_location=torch.device(device)))
# old_teacher_model.to(device)


num_epochs = 100
# Set teacher model
import itertools

train_loader = DataLoader(ds , batch_size = 8,shuffle=True , num_workers=4)

domainb_l_loader = DataLoader(domainb_train_l , batch_size=4 , shuffle=True,num_workers=4)
domainb_ul_loader = DataLoader(domainb_train_ul , batch_size=4 , shuffle=True,num_workers=4)

# cycle_loader = itertools.cycle(domainb_loader)


num_classes=7

In [ ]:
# prev_supernoise=None
# for i, data in enumerate(zip(domainb_l_loader ,domainb_ul_loader) ):

    
#     data1, data2 = data
#     images, masks,_,_ = data1
#     images, masks= images.to(device), masks.to(device)
#     outputs,_ = student_model(images.float())
#     outputs_soft = torch.softmax(outputs, dim=1)
    
    
    
#     unlabeled_images = data2[0]
#     unlabeled_images = unlabeled_images.to(device)
#     unlabeled_outputs ,_= student_model(unlabeled_images.float())
#     if prev_supernoise is not None:
#         noise = torch.mean(prev_supernoise, dim=1, keepdim=True)
#         target_size = unlabeled_images.shape[2:]
#         noise = F.interpolate(noise, size=target_size, mode='nearest')
#         noise=noise.expand(-1,3,-1,-1)
    
#     else:
#         noise = torch.clamp(torch.randn_like(unlabeled_images.float()) * 0.1, -0.2, 0.2)
        

#     ema_inputs = unlabeled_images + noise

#     with torch.no_grad():
#                 ema_output ,supernoise= teacher_model(ema_inputs.float())
#                 ema_output_soft = torch.softmax(ema_output, dim=1)
        
#         # Update the supernoise for the next iteration
#     prev_supernoise = supernoise
    
#     break

In [ ]:
# loss_dice = DiceLoss(n_classes=7,softmax=True,weight=[1,2,1,2,2,2,1])
max_iterations = 400 #close to few shot method 
iter_num = 0
ema_decay = 0.99

consistency_rampup = 200.0


In [ ]:
best_student_loss = 0.5

In [ ]:
epoch_data = {}  # Create an empty dictionary to store epoch data


for epoch in range(num_epochs):
    start_time = time.time()
    
    train_loss = train_mean2d(student_model, teacher_model, domainb_l_loader, domainb_ul_loader, loss_fn, optimizer, iter_num, scheduler,device)
    t_loss, t_f1, t_f1_class = validlossf2(teacher_model, domainb_test, loss_fn, num_classes=7)
    s_loss, s_f1, s_f1_class = validlossf2(student_model, domainb_test, loss_fn, num_classes=7)

    # Print the losses and F1 scores
    print(f"Epoch: {epoch+1:02} | Train Loss: {train_loss:.6f} | Teacher Loss: {t_loss:.6f} | Student Loss: {s_loss:.6f}")
    print(f"Teacher F1: {t_f1:.3f} | Student F1: {s_f1:.3f}")


    # if best_student_loss > s_loss:
    #     print(f"Student f1 improved from {best_student_loss:.6f} to {s_loss:.6f}. Saving checkpoint: {checkpoint_path}")
    #     best_student_loss = s_loss
    #     torch.save(student_model.state_dict(), "partitions/zero/5000.pth") #partitions/twentyfivr 

    end_time = time.time()
    epoch_mins, epoch_secs = divmod(end_time - start_time, 60)
    data_str = f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins:.0f}m {epoch_secs:.0f}s\n'
    data_str += f'\tTrain Loss: {train_loss:.6f} | Teacher Loss: {t_loss:.6f} | Student Loss: {s_loss:.6f}\n'
    data_str += f'\tTeacher F1: {t_f1:.3f} | Student F1: {s_f1:.3f}\n'
     # Append epoch data to the dictionary
    epoch_data[epoch+1] = {
        'train_loss': train_loss,
        'teacher_loss': t_loss,
        'student_loss': s_loss,
        'teacher_f1': t_f1,
        'student_f1': s_f1
    }
    # Print the epoch summary
    print(data_str)

In [ ]:
df = pd.DataFrame.from_dict(epoch_data, orient='index')


In [ ]:
df.to_excel("epoch_data_0.10.xlsx",index=False)

In [ ]:
#simple kd epoch 20 #lr0.001, epoch 9=0.777, epoch20=0.815 (many oscillations)---pefect config since lr change doesnt work 
#for above try initializing both simple networks of teachers weight 
#excel record asap, scehduler step, record the entire steps in excel 
# svd(student&teacher) 30 samelr,epoch=24 0.815  but all oscillatory --30 more epochs --0.826 at epoch 8 
#simple same weights(student and teacher ) 0.830 at 37 ,0.836 at 56







In [ ]:
#model names 
#studentsvdSW_efb4_257520.pth

In [ ]:
student_model.load_state_dict(torch.load( "studentsvdSW2_efb4_257520.pth"))
# old_teacher_model.load_state_dict(torch.load("efb4-full.pth"))

In [ ]:

ine,t,tt= validlossf2(student_model,domainb_test,loss_fn,num_classes)
tt

In [ ]:
t

In [ ]:
y=0
for x in tt :
    if x>0.9:
        pass
    else:
        y=y+x
        

In [ ]:
l,c,a,d = validlossf3(student_model,domainb_test,loss_fn,num_classes)


In [ ]:
d

In [ ]:
from matplotlib import pyplot as plt
# Assuming `d` is your DataFrame
d = d.iloc[:330, 1:]  # Ignore the first field and limit to first 330 rows

# Replace NaN values with 0
d = d.fillna(0)

# Create individual line plots for each field
for column in d.columns:
    plt.figure()  # Create a new figure for each plot
    plt.plot(d.index, d[column])
    plt.title(f"Line Plot for {column}")
    plt.xlabel("Row Number")
    plt.ylabel("Field Value")

# Show all the plots
plt.show()

In [ ]:
!pip install openpyxl



In [ ]:
d.to_excel("df.xlsx",index=False)

In [ ]:
d

In [ ]:
def validlossf3(model, ds, loss_fn, num_classes):
    f1_list = []
    loss_list = []
    
    model.eval()
    for img, mask, orig, _ in ds:
        img = img.to(device)
        mask = mask.to(device)
        with torch.no_grad():
            output = model(img.float().unsqueeze(0))
        loss = loss_fn(output, mask.float().unsqueeze(0))
        pred = torch.argmax(output, dim=1).squeeze(0)
        pred = pred.to("cpu")
        
        f1_scores = np.zeros(num_classes)
        
        for class_label in range(num_classes):  # F1 score and loss for the current class
            pred_class = (pred == class_label)
            real_class = (orig == class_label)
            
            true_positives = np.sum(np.logical_and(pred_class, real_class).numpy())
            false_positives = np.sum(np.logical_and(pred_class, ~real_class).numpy())
            false_negatives = np.sum(np.logical_and(~pred_class, real_class).numpy())
            
            precision = true_positives / (true_positives + false_positives)
            recall = true_positives / (true_positives + false_negatives)
            
            f1_scores[class_label] = 2 * (precision * recall) / (precision + recall)
        
        f1_list.append(np.round(list(f1_scores),5))
        loss_list.append(loss.item())
      
    average_loss = np.average(loss_list)
    f1df = pd.DataFrame(f1_list)
    f1avg = f1df[f1df != 0].mean()
    average_f1 = np.round(np.average(f1avg),5)

    return np.round(average_loss,7), np.round(average_f1,5), np.round(f1avg,5),f1df

In [ ]:
ine,t,tt= validlossf1(teacher_model,domainb_test,loss_fn,num_classes)
tt

In [ ]:
i,t,tt=validlossf2(old_teacher_model,domainb_test,loss_fn,num_classes)
t

In [ ]:
teacher_model1 = teacher_model1.to(device)


In [ ]:
len(domainb)

In [ ]:
teacher_model1 = smp.Unet(
    encoder_name="efficientnet-b4",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=7,                      # model output channels (number of classes in your dataset)
)
teacher_model1.load_state_dict(torch.load("efb4-full.pth" , map_location=torch.device(device)))

In [ ]:
teacher_model.load_state_dict(torch.load("newefb4svd-full.pth",map_location = torch.device(device)))


In [ ]:
i,t,tt=validlossf1(teacher_model,domainb_test,loss_fn,num_classes=7)
t

In [ ]:
i,t,tt=validlossf1(teacher_model1,domainb_train_ul,loss_fn,num_classes=7)
t

In [ ]:
i,t,tt=validlossf1(teacher_model1,domainb_train_l,loss_fn,num_classes=7)
t

In [ ]:
teacher_model.load_state_dict(torch.load("tea"))
i,t,tt=validlossf1(teacher_model,domainb_test,loss_fn,num_classes=7)
t